In [1]:
from automol.pipeline import Pipeline

In [2]:
config_yaml = 'qm9_dataset_test_config.yaml'
pipeline = Pipeline(config_yaml)
pipeline.print_spec()

amount: 10
dataset_class: QM9
dataset_location: dsgdb9nsd.xyz
labels:
- homo
models_to_exclude: []
problem: classification



In [3]:
pipeline.data_set.data['smiles'][0:10]

0    O=C([O-])C12C[NH2+]C1C2
1                CN1CC1C#CCO
2               OCC1COCOC=N1
3              COCCC(C#N)C=O
4            C1CC2OC2C2CN2C1
5           CC12C3OC1C2NC3=O
6                C#CCN(C)C=O
7            CC1OC2CC1C2(C)C
8              O=C1NC2CCC2O1
9          CC1(C(=O)C=O)CCC1
Name: smiles, dtype: object

In [4]:
pipeline.data_set.data['homo'][0:10]

0   -0.1795
1   -0.2284
2   -0.2521
3   -0.2701
4   -0.2201
5   -0.2270
6   -0.2516
7   -0.2310
8   -0.2575
9   -0.2452
Name: homo, dtype: float64

In [ ]:
from rdkit.Chem import Descriptors
from rdkit import Chem
from collections import OrderedDict
import inspect
import pandas as pd
import numbers

calc_props = OrderedDict(inspect.getmembers(Descriptors, inspect.isfunction))
smiles = pipeline.data_set.data['smiles']
sanitize = True
axis = 0

for key in list(calc_props.keys()):
    if key.startswith('_'):
        del calc_props[key]
        continue
    if key == 'setupAUTOCorrDescriptors':
        del calc_props[key]
        continue

df = pd.DataFrame(columns=list(calc_props.keys()), index=range(len(smiles)))

for i, s in enumerate(smiles):
    mol = Chem.MolFromSmiles(s)
    features = [val(mol) for key, val in calc_props.items()]
    non_number_features = [f for f in features if not isinstance(f, numbers.Number)]
    df.loc[i, :] = features

if sanitize:
    df.dropna(axis=axis, how='any', inplace=True)
    
non_number_features

In [ ]:
features

In [ ]:
pipeline.feature_generator.get_feature('fingerprint')

In [ ]:
pipeline.train()